In [1]:
# Setup the environment
!pip install -q --upgrade huggingface_hub
from huggingface_hub import login
# 登录到 Hugging Face Hub
login(token="填写你的huggingface token", add_to_git_credential=True)
!pip install -q transformers==4.38.0 accelerate==0.27.2 bitsandbytes==0.42.0 peft datasets==2.17.1 trl

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.12.2.post1 requires fsspec==2023.12.2, but you have fsspec 2023.10.0 which is incompatible.
s3fs 2024.2.0 requires fsspec==2024.2.0, but you have fsspec 2023.10.0 which is incompatible.


In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "/kaggle/input/gemma/transformers/2b-it/2"


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
LORA_WEIGHTS = "/kaggle/input/gemma-guzhenren/transformers/gemma/1/content/outputs"
model.eval()
model = PeftModel.from_pretrained(model, LORA_WEIGHTS)
print(model)
model = model.to("cuda")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Paramete

In [ ]:
test_prompt = '<|im_start|>user 你好<|im_end|>'
inp = tokenizer(test_prompt, max_length=512, return_tensors="pt").to("cpu")
outputs = model.generate(input_ids=inp["input_ids"], max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

In [13]:
test_prompt = '''
<|im_start|>user 
通过内容完善名为方源的角色刻画
你可以根据角色的实际情况，填写每个部分的相关信息，甚至对信息进行补充。这个模板可帮助你系统性地构建一个深度刻画的角色形象。


第二十二节：月刃飞舞
“月光蛊是我们古月一族的标志蛊虫，就像熊家的熊力蛊，白家的溪流蛊。在场的大多数人，都是选择月光蛊作为本命蛊，你们好好看着，接下来就老夫就亲自演示，如何催动月光蛊进行攻击。本命蛊不是月光蛊的学员，你们也要集中注意力，这种经典的远程攻击方法可以运用到其他蛊虫身上，运用范围非常广泛。”
“首先，用意念调动月光蛊，转移到自己的手心里。”随着他的话音，代表月光蛊的月牙印记，就顺着家老的手臂，移动到他的掌心中。
“然后，调动空窍中的真元，灌入到月光蛊中。”一丝白银色泽的真元，从家老的体内涌出，几乎细不可察，投入到掌心处的月光蛊中。
“太厉害了！”
“好美啊。”少年们目睹此景，不由地发出一阵惊奇赞叹。
学堂家老微微带笑：“都看仔细了，最后一步，就是像我这样，将它发射出去。”
“下面，只要是炼成了月光蛊的，都站出来。每人一个草人傀儡，按照我刚刚的方法，催发月刃，练习攻击。”
“哇，你看这记月牙！”
“好厉害，是谁发的？”
就连学堂家老的注意力，也被吸引过来，问道：“刚刚这记月牙发挥的不错，是你发的吗？”
“不过，看上去应该好像可能是的吧？”少年想着，就下意识地点点头。。
“他是谁呀，叫什么名字？”一些女学员纷纷向周围人打听。
“连他都能发出月刃，我更不能输！”古月漠北双眼闪过一丝坚定。
“原来不是哥哥发的。”古月方正莫名其妙地松了一口气。经过舅父舅母的安慰，他已经从前次的打击中恢复过来了。
“哥哥，你上次得了第一，是因为运气好，选了一个意志薄弱的月光蛊。蛊师修行不可能总依赖运道，我会赢你的。”方正在心中暗暗为自己打气。
“做的不错，要继续努力，抓住刚刚的那个感觉。”学堂家老拍拍这个学员的肩膀，微笑鼓励道。
家老借此机会，当众宣布：“都听好了，这就是你们的作业，课下好好练习，三天后检查成果。谁的成绩最好，谁就能获得十块元石奖励。都听到了吗？”
“哦！”少年们大声应喝着，听到有元石奖励，不禁更加兴奋。
“可恶，每一记月刃都要消耗掉一成的真元。”
“月刃的消耗太大了，我只是丙等修为，空窍中能存储三成八的青铜真元。只能连续催发出三片月刃。”
学堂家老目睹这一切，不动声色，却在心中感慨：“这就是修行资质高的好处了。要使用月刃，无非是四个字——熟能生巧。资质高的人，空窍中存储的真元就多，真元的恢复速度也快，就有更多的练习机会。资质若是差点，也可以用元石弥补，加强练习量。资质不高，又没有元石的，纵然有练习的想法，也是心有余而力不足。唉，蛊师修行就是这么残酷，我还是多多关照那些资质高的学生罢。”
<|im_end|>
'''

In [14]:
inp = tokenizer(test_prompt, max_length=2048, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inp["input_ids"], max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<bos>
<|im_start|>user 
通过内容完善名为方源的角色刻画
你可以根据角色的实际情况，填写每个部分的相关信息，甚至对信息进行补充。这个模板可帮助你系统性地构建一个深度刻画的角色形象。


第二十二节：月刃飞舞
“月光蛊是我们古月一族的标志蛊虫，就像熊家的熊力蛊，白家的溪流蛊。在场的大多数人，都是选择月光蛊作为本命蛊，你们好好看着，接下来就老夫就亲自演示，如何催动月光蛊进行攻击。本命蛊不是月光蛊的学员，你们也要集中注意力，这种经典的远程攻击方法可以运用到其他蛊虫身上，运用范围非常广泛。”
“首先，用意念调动月光蛊，转移到自己的手心里。”随着他的话音，代表月光蛊的月牙印记，就顺着家老的手臂，移动到他的掌心中。
“然后，调动空窍中的真元，灌入到月光蛊中。”一丝白银色泽的真元，从家老的体内涌出，几乎细不可察，投入到掌心处的月光蛊中。
“太厉害了！”
“好美啊。”少年们目睹此景，不由地发出一阵惊奇赞叹。
学堂家老微微带笑：“都看仔细了，最后一步，就是像我这样，将它发射出去。”
“下面，只要是炼成了月光蛊的，都站出来。每人一个草人傀儡，按照我刚刚的方法，催发月刃，练习攻击。”
“哇，你看这记月牙！”
“好厉害，是谁发的？”
就连学堂家老的注意力，也被吸引过来，问道：“刚刚这记月牙发挥的不错，是你发的吗？”
“不过，看上去应该好像可能是的吧？”少年想着，就下意识地点点头。。
“他是谁呀，叫什么名字？”一些女学员纷纷向周围人打听。
“连他都能发出月刃，我更不能输！”古月漠北双眼闪过一丝坚定。
“原来不是哥哥发的。”古月方正莫名其妙地松了一口气。经过舅父舅母的安慰，他已经从前次的打击中恢复过来了。
“哥哥，你上次得了第一，是因为运气好，选了一个意志薄弱的月光蛊。蛊师修行不可能总依赖运道，我会赢你的。”方正在心中暗暗为自己打气。
“做的不错，要继续努力，抓住刚刚的那个感觉。”学堂家老拍拍这个学员的肩膀，微笑鼓励道。
家老借此机会，当众宣布：“都听好了，这就是你们的作业，课下好好练习，三天后检查成果。谁的成绩最好，谁就能获得十块元石奖励。都听到了吗？”
“哦！”少年们大声应喝着，听到有元石奖励，不禁更加兴奋。
“可恶，每一记月刃都要消耗掉一成的真元。”
“月刃的消耗太大了，我只是丙等修为，空窍中能存储三成八的青铜真元。只能连续催发出三片月刃。”
学堂家老目睹这一切，不